### Importing Libraries and Defining Paths

In [ ]:
from warnings import filterwarnings
filterwarnings(action='ignore', category=DeprecationWarning, message='`np.bool` is a deprecated alias')
filterwarnings('ignore')

In [ ]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import fiona 
from shapely.geometry import shape, box
import rasterio
from PIL import Image, ImageDraw
Image.MAX_IMAGE_PIXELS = None
import tensorflow as tf
import imgaug.augmenters as iaa
import matplotlib.pyplot as plt
%matplotlib inline 
from patchify import patchify, unpatchify
from shapely.geometry import Polygon, MultiPolygon
import geopandas as gpd
import copy

In [13]:
# Root directory of the project
ROOT_DIR = os.path.abspath("../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
sys.path.append(os.path.join(ROOT_DIR, "mrcnn"))

In [14]:
from mrcnn.config import Config
from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

In [15]:
# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

DATASET_DIR = os.path.join(ROOT_DIR, "Dataset")

# Local path to trained weights file
COCO_MODEL_PATH = "../models/mask_rcnn_coco.h5"
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

In [16]:
# Check if GPU is available
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

### Configurations

In [17]:
class TrainingConfig(Config):
    NAME = "BuildingDetection"
    GPU_COUNT = 1
    IMAGES_PER_GPU = 4
    STEPS_PER_EPOCH = 2770
    VALIDATION_STEPS = 234
    BACKBONE = "resnet101"
    NUM_CLASSES = 2 #bulding and background
    IMAGE_MAX_DIM = 1024
    IMAGE_MIN_DIM = 1024

    TRAIN_ROIS_PER_IMAGE = 50
    
    MAX_GT_INSTANCES = 50
    LOSS_WEIGHTS = {
        "rpn_class_loss": 2.,
        "rpn_bbox_loss": 1.,
        "mrcnn_class_loss": 1.,
        "mrcnn_bbox_loss": 1.,
        "mrcnn_mask_loss": 10.
    }

In [18]:
class InferenceConfig(TrainingConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    DETECTION_MIN_CONFIDENCE = 0.9

inference_config = InferenceConfig()

In [19]:
model_inference = modellib.MaskRCNN(mode="inference", config=inference_config, model_dir=MODEL_DIR)

### Inference

In [20]:
# Get path to saved weights
# Either set a specific path or find last trained weights
model_path = os.path.join(MODEL_DIR, "buildingdetection20230801T1248/mask_rcnn_buildingdetection_0041.h5")
# model_path = model_inference.find_last()

# Load trained weights
print("Loading weights from ", model_path)
model_inference.load_weights(model_path, by_name=True)

Loading weights from  /home/cctv/Internship/Building_detection/logs/buildingdetection20230801T1248/mask_rcnn_buildingdetection_0041.h5


2023-09-24 17:40:31.634260: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-24 17:40:31.641806: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:51:00.0 name: NVIDIA A30 computeCapability: 8.0
coreClock: 1.44GHz coreCount: 56 deviceMemorySize: 23.50GiB deviceMemoryBandwidth: 869.04GiB/s
2023-09-24 17:40:31.646473: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1871] Adding visible gpu devices: 0
2023-09-24 17:40:31.646542: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2023-09-24 17:40:31.817110: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1258] Device interconnect StreamEx

Re-starting from epoch 41


#### Run inference on multiple files

In [21]:
def extract_patches(image_path, patch_size, step):
    """
    Extract patches from an image and return the patches along with a padded image.

    Args:
        image_path (str): Path to the input image.
        patch_size (tuple): Size of each patch in (height, width) format.
        step (int): Step size for patch extraction.

    Returns:
        patches (numpy.ndarray): Extracted patches.
        padded_image (numpy.ndarray): Padded image to fit patches evenly.
    """
    # Open the image
    image = Image.open(image_path)
    
    # Convert the PIL image to a NumPy array
    image_np = np.array(image)
    
    # Calculate the number of patches in each dimension
    num_patches_y = (image_np.shape[0] + patch_size[0] - 1) // patch_size[0]
    num_patches_x = (image_np.shape[1] + patch_size[1] - 1) // patch_size[1]
    
    # Calculate the required padding to fit the patches evenly
    pad_y = num_patches_y * patch_size[0] - image_np.shape[0]
    pad_x = num_patches_x * patch_size[1] - image_np.shape[1]
    
    # Pad the image
    
    if(len(image_np.shape)<3):
        print("RGB channels missing in", image_path)
        return [], []
    else:
        padded_image = np.pad(image_np, ((0, pad_y), (0, pad_x), (0, 0)), mode='constant')
    
    # Divide the padded image into patches
    patches = patchify(padded_image, patch_size, step=step)
    
    return patches, padded_image

In [22]:
def is_mask_on_edge(mask, edge_percentage, mask_percentage):
    """
    Check if a mask is present on the edges of an image.

    Args:
        mask (numpy.ndarray): Binary mask.
        edge_percentage (float): Percentage of edges to consider (0 to 1).
        mask_percentage (float): Minimum percentage of mask pixels on an edge (0 to 1).

    Returns:
        bool: True if mask is on any edge, False otherwise.
    """
    height, width = mask.shape
    total_mask_area = np.sum(mask)
    edge_pixels = int(min(height, width) * edge_percentage)

    # Check top edge
    if np.sum(mask[:edge_pixels, :]) >= mask_percentage * total_mask_area:
        return True
    
    # Check bottom edge
    if np.sum(mask[-edge_pixels:, :]) >= mask_percentage * total_mask_area:
        return True
    
    # Check left edge
    if np.sum(mask[:, :edge_pixels]) >= mask_percentage * total_mask_area:
        return True
    
    # Check right edge
    if np.sum(mask[:, -edge_pixels:]) >= mask_percentage * total_mask_area:
        return True
    
    return False

In [23]:
def find_polygons(r, x, y, eps = 0.02):
    """
    Find polygons from instance segmentation results.

    Args:
        r (dict): Result dictionary containing masks and bounding boxes.
        x (int): X-coordinate shift.
        y (int): Y-coordinate shift.
        eps (float, optional): Epsilon value for polygon approximation. 
            Controls the precision of the approximation. Default is 0.02.

    Returns:
        polygons (list): List of Shapely Polygon objects representing objects.
    """
    masks = r['masks']
    bboxes = r['rois']
    n = bboxes.shape[0]
    polygons = []
    
    for i in range(n):
        mask = masks[:, :, i]
        mask = mask.astype(np.uint8)

        # Ignore incomplete building masks on sides of images
        if is_mask_on_edge(mask, 0.2, 0.5):
            continue
        
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        for contour in contours:
            epsilon = eps * cv2.arcLength(contour, True)
            approx = cv2.approxPolyDP(contour, epsilon, closed=False)
            
            shifted_polygon = np.array(approx) + np.array([x, y])
            
            # Convert the shifted_polygon to a GeoDataFrame Polygon
            poly_coords = [(point[0][0], point[0][1]) for point in shifted_polygon]
            if len(poly_coords) >= 4:
                polygon = Polygon(poly_coords)
                polygons.append(polygon)
            
    return polygons

In [24]:
def apply_patch_overlap_adjustment(polygons, prev_polygons, overlap_threshold):
    adjusted_polygons = []
    for polygon in polygons:
        if (polygon.is_valid and prev_poly.is_valid() and not any(polygon.intersection(prev_poly).area >= overlap_threshold * polygon.area) for prev_poly in prev_polygons):
            adjusted_polygons.append(polygon)
    return adjusted_polygons

In [25]:
def process_patches_with_overlap_adjustment(image_path, patches, model_inference, step, overlap_threshold, output_shp_path):
    # Open the input image
    image = Image.open(image_path)
    image_width, image_height = image.size
    draw = ImageDraw.Draw(image)

    # Extract CRS from the TIFF image
    with rasterio.open(image_path) as src:
        transform = src.transform
        crs = src.crs
    
    # Initialize a 2D list to store polygons for each patch
    patch_polygons = [[[] for _ in range(patches.shape[1])] for _ in range(patches.shape[0])]
    
    # Create a list to store all polygons for the shapefile
    all_polygons = []

    for i in range(patches.shape[0]):
        for j in range(patches.shape[1]):
            # Extract the current patch
            patch = patches[i, j, 0, :, :, :3]
            
            # Perform model inference to detect objects in the patch
            r = model_inference.detect([patch])[0]
            
            # Find polygons from the detection results and adjust for overlap
            polygons = find_polygons(r, j * step, i * step)
            
            # Apply overlap adjustment with the previous row
            if i > 0:
                prev_row_polygons = patch_polygons[i - 1][j]
                polygons = apply_patch_overlap_adjustment(polygons, prev_row_polygons, overlap_threshold)
            
            # Apply overlap adjustment with the previous column
            if j > 0:
                prev_col_polygons = patch_polygons[i][j - 1]
                polygons = apply_patch_overlap_adjustment(polygons, prev_col_polygons, overlap_threshold)
            
            # Apply overlap adjustment with the diagonal patch (top-left)
            if i > 0 and j > 0:
                prev_diag_polygons = patch_polygons[i - 1][j - 1]
                polygons = apply_patch_overlap_adjustment(polygons, prev_diag_polygons, overlap_threshold)
            
            # Apply overlap adjustment with the diagonal patch (top-right)
            if i > 0 and j < patches.shape[1]-1:
                prev_diag_polygons = patch_polygons[i - 1][j + 1]
                polygons = apply_patch_overlap_adjustment(polygons, prev_diag_polygons, overlap_threshold)
            
            # Store the adjusted polygons in the 2D list
            patch_polygons[i][j] = polygons

            # Append the adjusted polygons to the list of all polygons
            all_polygons.extend(polygons)
            
            # Print a message to indicate patch processing completion
            print(f"Patch {i},{j} completed")

    # Transform polygon coordinates from pixel to the TIFF file's coordinate system
    transformed_polygons = []
    for polygon in all_polygons:
        points = []
        for point in polygon.exterior.coords:
            x, y = point
            lon, lat = transform * (x + 0.5, y + 0.5)
            points.append((lon, lat))
        transformed_polygons.append(Polygon(points))

    # Create a GeoDataFrame from the list of transformed polygons
    geometry = gpd.GeoSeries(transformed_polygons)
    print(len(all_polygons))
    gdf = gpd.GeoDataFrame(geometry=geometry, crs=crs)  # You might need to adjust the CRS
    
    # Save the GeoDataFrame as a shapefile
    gdf.to_file(output_shp_path)

In [32]:
overlap_threshold = 0.5  # Adjust this threshold as needed
patch_size = (1024, 1024, 3)  # Adjust this to the desired patch size and channel count
step = 512
input_folder = "../Dataset/Orthos_DSMs_Shapefiles/"
output_folder = "../results"

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Iterate over folders in the input directory
for tif_folder in os.listdir(input_folder):
    geotiff_folder = os.path.join(input_folder, tif_folder)
    geotiff_folder = os.path.join(geotiff_folder, "GeoTiff2")
    curr_output_folder = os.path.join(output_folder, tif_folder)
    
    # Create output folder if it doesn't exist
    if not os.path.exists(curr_output_folder):
        os.makedirs(curr_output_folder)

    # Iterate over files in the GeoTiff folder
    for filename in os.listdir(geotiff_folder):    
        if filename.endswith(".tif"):
            input_path = os.path.join(geotiff_folder, filename)
            print("Working on", input_path)

            inside_folder = os.path.join(curr_output_folder, f"{filename[:-4]}")
            
            # Create a subfolder for each input file
            if not os.path.exists(inside_folder):
                os.makedirs(inside_folder)

            # if len(os.listdir(inside_folder))==0:
            #     os.rmdir(inside_folder)
            #     print("deleting", inside_folder)
            #     continue
 
            output_path = os.path.join(inside_folder, f"output_{filename[:-4]}"+"_palette_file.shp")
            print("Output path:", output_path)
            
            # Skip processing if the output file already exists
            if os.path.exists(output_path):
                continue
            
            # Extract patches from the input image
            patches, padded_image = extract_patches(input_path, patch_size, step)\
            
            # Process patches with overlap adjustment and save results as a shapefile
            if(len(patches)):
                process_patches_with_overlap_adjustment(input_path, patches, model_inference, step, overlap_threshold, output_path)

Working on ../Dataset/Orthos_DSMs_Shapefiles/Dharmavaram_Dharmavaram_Elukuntla_2617200/GeoTiff2/Elukuntala_Ortho_H2.tif
Output path: ../results/Dharmavaram_Dharmavaram_Elukuntla_2617200/Elukuntala_Ortho_H2/output_Elukuntala_Ortho_H2_palette_file.shp
Patch 0,0 completed
Patch 0,1 completed
Patch 0,2 completed
Patch 0,3 completed
Patch 0,4 completed
Patch 0,5 completed
Patch 0,6 completed
Patch 0,7 completed
Patch 0,8 completed
Patch 0,9 completed
Patch 0,10 completed
Patch 0,11 completed
Patch 0,12 completed
Patch 0,13 completed
Patch 0,14 completed
Patch 1,0 completed
Patch 1,1 completed
Patch 1,2 completed
Patch 1,3 completed
Patch 1,4 completed
Patch 1,5 completed
Patch 1,6 completed
Patch 1,7 completed
Patch 1,8 completed
Patch 1,9 completed
Patch 1,10 completed
Patch 1,11 completed
Patch 1,12 completed
Patch 1,13 completed
Patch 1,14 completed
Patch 2,0 completed
Patch 2,1 completed
Patch 2,2 completed
Patch 2,3 completed
Patch 2,4 completed
Patch 2,5 completed
Patch 2,6 completed


#### Run inference on single file

In [ ]:
# overlap_threshold = 0.5  # Adjust this threshold as needed
# image_path = "../Dataset/test_files/Chowtakuntapalli_ORTHO_H1.tif"  # Path to your test image
# patch_size = (1024, 1024, 3)  # Adjust this to the desired patch size and channel count
# step = 512

In [ ]:
# patches, padded_image = extract_patches(image_path, patch_size, step)
# print(f"Number of patches: {patches.shape[0] * patches.shape[1]}")
# print(f"Shape of a single patch: {patches[0, 0, 0].shape}")

In [ ]:
# save_patches(patches, "../Dataset/patches")

In [ ]:
# output_shp_path = "../shp_files/output_shp.shp"
# process_patches_with_overlap_adjustment(image_path, patches, model_inference, step, overlap_threshold, output_shp_path)